In [1]:
import pandas
import os
import datetime
import time
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .enableHiveSupport() \
    .getOrCreate()

# dimensao_calendario

In [4]:
for i in range(365):
    d = datetime.datetime(2021,1,1) + datetime.timedelta(days=i)
    spark.sql("""
        insert into dimensao_calendario partition (
            ano = {2},
            mes = {3}
        ) values (
            {0},
            date '{1}',
            {4}
        )
    """.format(d.strftime('%Y%m%d'), d.strftime('%Y-%m-%d'), d.strftime('%Y'), int(d.strftime('%m')), int(d.strftime('%d'))))

# dimensao_semana_ano_iso

In [5]:
data = []
for i in range(365):
    d = datetime.datetime(2021,1,1) + datetime.timedelta(days=i)
    if int(d.strftime('%G%V')) not in data:
        data.append(int(d.strftime('%G%V')))
        spark.sql("""
            insert into dimensao_semana_ano_iso partition (
                ano_iso = {0}
            ) values (
                {1},
                {2},
                '{3}'
            )
        """.format(int(d.strftime('%G')), int(d.strftime('%G%V')), int(d.strftime('%V')), d.strftime('%G-S%V')))
del data

# dimensao_mes

In [6]:
data = []
for i in range(365):
    d = datetime.datetime(2021,1,1) + datetime.timedelta(days=i)
    if int(d.strftime('%Y%m')) not in data:
        data.append(int(d.strftime('%Y%m')))
        spark.sql("""
            insert into dimensao_mes partition (
                ano = {0}
            ) values (
                {1},
                {2},
                '{3}'
            )
        """.format(int(d.strftime('%Y')), int(d.strftime('%Y%m')), int(d.strftime('%m')), d.strftime('%b/%Y')))
del data

# dimensao_eps

In [10]:
df = pandas.read_excel('exemplos.xlsx', sheet_name='dimensao_eps', header=0)
for i, r in df.iterrows():
    spark.sql("""
        insert into dimensao_eps values ({0}, '{1}', '{2}')
    """.format(r['sk_eps'], r['nome_eps'], r['sigla_eps']))
del df

# dimensao_celula

In [11]:
df = pandas.read_excel('exemplos.xlsx', sheet_name='dimensao_celula', header=0)
for i, r in df.iterrows():
    spark.sql("""
        insert into dimensao_celula values ({0}, '{1}')
    """.format(r['sk_celula'], r['nome_celula']))
del df

In [ ]:
spark.stop()
del spark